<a href="https://colab.research.google.com/github/sushantachary/CPT_S-437/blob/main/Predicting_Subscription_Project_CPTS437.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# Upload the kaggle.json file
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sushantacharya","key":"973bf1322838c1227f1a6b5291f4cfce"}'}

In [2]:
# Move the uploaded kaggle.json file to the .kaggle directory
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# Install Kaggle and other required libraries
!pip install kaggle pandas numpy scikit-learn matplotlib seaborn

In [4]:
# Use Kaggle API to download the Telco Customer Churn dataset
!kaggle datasets download -d blastchar/telco-customer-churn

Dataset URL: https://www.kaggle.com/datasets/blastchar/telco-customer-churn
License(s): copyright-authors
  0% 0.00/172k [00:00<?, ?B/s]
100% 172k/172k [00:00<00:00, 63.8MB/s]


In [7]:
# Unzip the downloaded dataset
!unzip telco-customer-churn.zip

Archive:  telco-customer-churn.zip
replace WA_Fn-UseC_-Telco-Customer-Churn.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: WA_Fn-UseC_-Telco-Customer-Churn.csv  


In [8]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix


# Load the dataset
data = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Display the first few rows of the dataset
print(data.head())

# Check for missing values and data types
print(data.info())

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract Pape

**Random** forest:

In [9]:
# Data Preprocessing
# Drop 'customerID' column as it's irrelevant
data.drop('customerID', axis=1, inplace=True)

# Convert 'TotalCharges' to numeric (handle errors) and fill missing values
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')
data['TotalCharges'].fillna(data['TotalCharges'].median(), inplace=True)

# Encode target variable 'Churn' (Yes -> 1, No -> 0)
data['Churn'] = data['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

# Convert categorical variables to dummy variables
categorical_columns = data.select_dtypes(include=['object']).columns
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Split data into features (X) and target (y)
X = data.drop('Churn', axis=1)  # Features
y = data['Churn']  # Target

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

predicted_output = pd.DataFrame({
    'Actual': y_test.values,      # The true labels
    'Predicted': y_pred  # The predicted labels
}).reset_index(drop=True)

# Display the first few rows of the results
print(predicted_output)


<ipython-input-9-25138fc519fc>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['TotalCharges'].fillna(data['TotalCharges'].median(), inplace=True)


Model Accuracy: 0.7864
      Actual  Predicted
0          0          0
1          0          1
2          0          0
3          0          0
4          0          0
...      ...        ...
1404       0          0
1405       0          0
1406       0          0
1407       0          0
1408       0          0

[1409 rows x 2 columns]


Gradient Boosting Machines:

In [10]:
# Load the dataset
data = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Data Preprocessing
# Drop 'customerID' column
data.drop('customerID', axis=1, inplace=True)

# Convert 'TotalCharges' to numeric and handle missing values
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')
data['TotalCharges'].fillna(data['TotalCharges'].median(), inplace=True)

# Encode target variable 'Churn'
data['Churn'] = data['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

# Convert categorical variables to dummy variables
categorical_columns = data.select_dtypes(include=['object']).columns
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Separate features and target variable
X = data.drop('Churn', axis=1)
y = data['Churn']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train the GBM model
gbm_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbm_model.fit(X_train, y_train)

# Chunk-Based Predictions
chunk_size = 500  # Process data in chunks of 500 rows
num_chunks = (len(X_test) + chunk_size - 1) // chunk_size  # Calculate number of chunks
chunk_predictions = []

for i in range(num_chunks):
    start = i * chunk_size
    end = min(start + chunk_size, len(X_test))
    chunk = X_test[start:end]
    predictions = gbm_model.predict(chunk)
    chunk_predictions.extend(predictions)

# Evaluate the model
accuracy = accuracy_score(y_test, chunk_predictions)
print(f"Model Accuracy (Chunk-Based): {accuracy:.4f}")

predicted_output = pd.DataFrame({
    'Actual': y_test.values,      # The true labels
    'Predicted': chunk_predictions  # The predicted labels
}).reset_index(drop=True)

# Display the first few rows of the results
print(predicted_output)

<ipython-input-10-57c60e7387f3>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['TotalCharges'].fillna(data['TotalCharges'].median(), inplace=True)


Model Accuracy (Chunk-Based): 0.7991
      Actual  Predicted
0          0          0
1          0          1
2          0          0
3          0          0
4          0          0
...      ...        ...
1404       0          0
1405       0          0
1406       0          0
1407       0          0
1408       0          0

[1409 rows x 2 columns]


Logistic Regression:

In [11]:
# Load the dataset
data = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Data Preprocessing
# Drop 'customerID' column
data.drop('customerID', axis=1, inplace=True)

# Convert 'TotalCharges' to numeric and handle missing values
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')
data['TotalCharges'].fillna(data['TotalCharges'].median(), inplace=True)

# Encode target variable 'Churn'
data['Churn'] = data['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

# Convert categorical variables to dummy variables
categorical_columns = data.select_dtypes(include=['object']).columns
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Separate features and target variable
X = data.drop('Churn', axis=1)
y = data['Churn']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train the Logistic Regression model
log_reg_model = LogisticRegression(max_iter=5000, random_state=42)
log_reg_model.fit(X_train, y_train)

# Make predictions
y_pred = log_reg_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

# Create a DataFrame to display actual vs predicted results
predicted_output = pd.DataFrame({
    'Actual': y_test.values,      # The true labels
    'Predicted': y_pred           # The predicted labels
}).reset_index(drop=True)

# Display the predicted output
print(predicted_output)


<ipython-input-11-dfc604a68ebb>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['TotalCharges'].fillna(data['TotalCharges'].median(), inplace=True)


Model Accuracy: 0.8062
      Actual  Predicted
0          0          0
1          0          1
2          0          0
3          0          0
4          0          0
...      ...        ...
1404       0          0
1405       0          0
1406       0          0
1407       0          0
1408       0          0

[1409 rows x 2 columns]


In [12]:
models = {
    "Radom Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=5000, random_state=42)
}

results = {}

for model_name, model in models.items():
  print(f"Training and evaluating: {model_name}")
  start_time = time.time()
  model.fit(X_train, y_train)
  end_time = time.time()
  processing_time = end_time - start_time
  print(f"Processing time for {model_name}: {processing_time:.2f} seconds")
  y_pred = model.predict(X_test)
  # Check if the model has predict_proba and if it returns probabilities for more than one class
  if hasattr(model, "predict_proba") and model.predict_proba(X_test).shape[1] > 1:
    y_prob = model.predict_proba(X_test)[:, 1]
  else:
    y_prob = None  # or handle it differently, e.g., use decision function for binary classification

  acc = accuracy_score(y_test, y_pred)
  prec = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  auc = roc_auc_score(y_test, y_prob) if y_prob is not None else "n/a"

  results[model_name] = {
      "Accuracy": acc,
      "Precision": prec,
      "Recall": recall,
      "F1 Score": f1,
      "AUC": auc
  }

  print(f"Classification Report for {model_name}: \n")
  print(classification_report(y_test, y_pred))
  print("-" * 50)

results_df = pd.DataFrame(results).T
print("\nComparison of Model Performance: ")
print(results_df)

Training and evaluating: Radom Forest
Processing time for Radom Forest: 0.75 seconds
Classification Report for Radom Forest: 

              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1035
           1       0.62      0.49      0.55       374

    accuracy                           0.79      1409
   macro avg       0.73      0.69      0.71      1409
weighted avg       0.77      0.79      0.78      1409

--------------------------------------------------
Training and evaluating: Gradient Boosting
Processing time for Gradient Boosting: 1.10 seconds
Classification Report for Gradient Boosting: 

              precision    recall  f1-score   support

           0       0.84      0.90      0.87      1035
           1       0.65      0.52      0.58       374

    accuracy                           0.80      1409
   macro avg       0.75      0.71      0.72      1409
weighted avg       0.79      0.80      0.79      1409

--------------------------